In [1]:
import pandas as pd
import re
import pickle
import datetime

In [2]:
with open('log.pickle','rb') as f:
    df = pickle.load(f)

In [3]:
df['log_date'] = df.log_date.apply(lambda x: datetime.datetime.strptime(x, "%d/%b/%Y:%H:%M:%S"))

df = df[df.device_info != 'Amazon CloudFront']
df.drop('device_info', axis = 1, inplace = True)

df = df.sort_values(by = 'log_ip')
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace = True)

df = df.sort_values(by=['log_ip','log_date'])
df.reset_index(inplace = True)
df.drop('index', axis = 1, inplace = True)

In [4]:
#보안 문제로 IP주소는 삭제함.
df.drop('log_ip', axis = 1).head(3)

,log_date,before_url,after_url,device,os,app
0,2018-07-16 16:18:18,http://m.mail.daum.net/hanmailex/mobile/Top.da...,https://www.shopintake.com/event/view/139/?utm...,Mobile,iPhone,Web
1,2018-07-16 16:19:05,https://www.shopintake.com/event/view/139/?utm...,https://www.shopintake.com/event/,Mobile,iPhone,Web
2,2018-07-16 16:19:27,https://www.shopintake.com/event/,https://www.shopintake.com/event/?page=2,Mobile,iPhone,Web


In [5]:
df['sign_up'] = df.after_url
df['log_in'] = df.after_url
df['pay_complete'] = df.after_url
df['sign_up'] = df.sign_up.apply(lambda x: 100 if re.compile('/membership/signup/complete').search(x) else 0)
df['log_in'] = df.log_in.apply(lambda x: 10 if re.compile('/membership/login').search(x) else 0)
df['pay_complete'] = df.pay_complete.apply(lambda x: 1 if re.compile('/payment/complete').search(x) else 0)

In [6]:
df_total = pd.DataFrame(columns = ['log_ip','log_date','sign_up','log_in', 'pay_complete'])
df_total['log_ip'] = df.groupby(['log_ip']).apply(list).index
df_total['log_date'] = df.groupby(['log_ip'])['log_date'].apply(list).values

df_total['sign_up'] = df.groupby(['log_ip'])['sign_up'].apply(list).values
df_total['log_in'] = df.groupby(['log_ip'])['log_in'].apply(list).values
df_total['pay_complete'] = df.groupby(['log_ip'])['pay_complete'].apply(list).values

df_total['url_cnt'] = df_total.log_date.apply(lambda x: len(x))

In [7]:
# 한 세션의 기준은 30분으로 한다.
df_total['split_index'] = df_total.log_date.apply(lambda x: [i for i in range(1,len(x)) if (x[i] - x[i-1]).seconds//60 > 30])

In [8]:
df_total.shape

(29117, 7)

In [9]:
def coder(x_list):
    r_list = list()
    for x in x_list:
        if (x//100 > 1) and (x%100 >= 10) and (x%10 >= 1) :
            r_list.append('lb')
        elif (x%100 >= 10) and (x%10 >=1):
            r_list.append('lb')
        elif (x//100 > 1) and (x%100 >= 10) and (x%10 == 0):
            r_list.append('lnb')
        elif (x%100 >= 10) and (x%10 == 0):
            r_list.append('lnb')
        else : 
            r_list.append('dn')
    if 'lb' in r_list:
        return 'lb'
    elif 'lnb' in r_list:
        return 'lnb'
    else : 
        return 'dn'

In [10]:
login_buy = list()
login_nobuy = list()
do_nothing = list()

for _ in range(df_total.shape[0]):
    user_type = list()
    temp_df = df_total.iloc[_]
    if len(temp_df.split_index) == 0:
        c = coder([i+j for i,j in zip([i+j for i,j in zip(temp_df.sign_up, temp_df.log_in)], temp_df.pay_complete)])
        if c == 'lb':
            login_buy.append(temp_df.log_ip)
        elif c =='lnb':
            login_nobuy.append(temp_df.log_ip)
        else:
            do_nothing.append(temp_df.log_ip)
            continue
        
        
    elif len(temp_df.split_index) == 1:
        for i in range(len(temp_df.split_index)):
            if i == 0:
                idx = temp_df.split_index[0]
                c = coder([i+j for i,j in zip([i+j for i,j in zip(temp_df.sign_up[:idx], temp_df.log_in[:idx])], temp_df.pay_complete[:idx])]\
                +[i+j for i,j in zip([i+j for i,j in zip(temp_df.sign_up[idx:], temp_df.log_in[idx:])], temp_df.pay_complete[idx:])])
                if c == 'lb':
                    login_buy.append(temp_df.log_ip)
                elif c =='lnb':
                    login_nobuy.append(temp_df.log_ip)
                else:
                    do_nothing.append(temp_df.log_ip)
                continue
    else:
        for i in range(len(temp_df.split_index)):
            if i == 0:
                idx = temp_df.split_index[i]
                c_ = [i+j for i,j in zip([i+j for i,j in zip(temp_df.sign_up[:idx], temp_df.log_in[:idx])], temp_df.pay_complete[:idx])]
                continue
            elif i < len(temp_df.split_index)-1:
                idx = temp_df.split_index[i]
                c_ = c_ + [i+j for i,j in zip([i+j for i,j in zip(temp_df.sign_up[idx-1:idx], temp_df.log_in[idx-1:idx])], temp_df.pay_complete[idx-1:idx])]
                continue
            elif i == len(temp_df.split_index)-1:
                idx = temp_df.split_index[i]
                c_ = c_ + [i+j for i,j in zip([i+j for i,j in zip(temp_df.sign_up[idx-1:], temp_df.log_in[idx-1:])], temp_df.pay_complete[idx-1:])]
            c = coder(c_)
            if c == 'lb':
                login_buy.append(temp_df.log_ip)
            elif c =='lnb':
                login_nobuy.append(temp_df.log_ip)
            else:
                do_nothing.append(temp_df.log_ip)

In [11]:
print(len(login_buy),   # 한 세션에서 로그인하고 상품을 구매한 사람 ( + 회원가입)
      len(login_nobuy), # 한 세션에서 로그인하고 상품을 구매하지 않은 사람 (+ 회원가입)
      len(do_nothing)   # 그 외
     )

43 3137 25937


In [12]:
ip_check_buy = {
    'login_buy' : login_buy,
    'login_nobuy' : login_nobuy,
    'do_nothing' : do_nothing
}

In [13]:
with open('ip_check_buy.pickle', 'wb') as f:
    pickle.dump(ip_check_buy, f)